In [ ]:
!apt install -qq enchant
!pip install pyenchant
!pip install wordcloud
!pip install imblearn

In [ ]:
import pandas as pd
import gzip
import json
import numpy as np
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import dask 
import dask.dataframe as dd
import imblearn
from collections import Counter

import os
import sys
import math
import time
import nltk
import torch
import random
import string
import unidecode
import collections

import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.model_selection import train_test_split

import imblearn
from imblearn.under_sampling import RandomUnderSampler

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import enchant
from enchant.checker import SpellChecker

from autocorrect import Speller
import dask
import dask.dataframe as dd

import time

%matplotlib inline

In [ ]:
# read electronics from json.gz
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

elec = getDF('Electronics_5_2018.json.gz')

#We don't have much use for any records without reviews
elec = elec[elec['reviewText'].notna()]

In [ ]:
# Read all_category data
allcats = pd.read_csv('all_cats.csv')

In [ ]:
def detect_gib(sentence):
    len_sent = len(sentence.split())
    chkr = SpellChecker("en_US")

    chkr.set_text(sentence)
    num_errors = 0
    for err in chkr:
        num_errors = num_errors + 1
    if num_errors > len_sent / 2:
        return None
    else:
        return sentence


#spellcheck

check = Speller(lang = 'en')

def fixSentence(sentence):
    return check(sentence)





#Create a dictionary of english contractions and their expanded forms.
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

# Regular expression for finding contractions
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contraction(text, contractions_dict = contractions_dict):
    
    #for anything that matches the re we defined earlier for the contraction keys, replace with the value
    
    def replace(match):
        return contractions_dict[match.group(0)]
    
    return contractions_re.sub(replace, text)

def clean_text(text):
    
    # make sure the text is of type str if not for some weird reason, then set to lowercase. 
    text = str(text).lower()
    
    #Spell check
    #text = fixSentence(text)
    
    #Expand contractions
    text = expand_contraction(text)
    
    #Remove links. If any text starts with either http://, https://, or www., replace it with a space
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    #Clear newline chars
    text = re.sub('\n', ' ', text)
    
    #Add spaces before and after certain punctuation so we can tokenize it. 
    #We thought about using the + chararcter after everything, but embeddings for sequences like ?! are probably better off split up
    #We only included it for periods so we can capture elipses, which have distinct meanings from periods.
    text = re.sub(r"[.]+|[,;!?]"," \g<0> ", text)
    
    #change & to and
    text = re.sub(r"[&]"," and ", text)
    
    #change _ to a space
    text = re.sub(r"[_]"," ", text)
    
    #get rid of any character not in this list
    text = re.sub(r"[^a-zA-Z0-9.,;!?\s/]","", text)
    
    #When it comes time to vectorize, we'll exclude unwanted symbols by using a regexpTokenizer and whitelisting approved punctuation
    #token = RegexpTokenizer(r"[a-zA-Z0-9.,;!?]+")
    
    return text

In [ ]:
def preprocess(df, saved_name):
    #get rid of the style and image columns, which are not very useful and rather unwieldy
    df.drop(['style', 'image'], axis=1, inplace =True) 
    
    # create dask df to parallelize
    dask_df = dd.from_pandas(df, npartitions = 16)
    
    #detect gibberish. Get rid of recors that are half or more gibberish
    dask_df['reviewText'] = dask_df['reviewText'].apply(detect_gib, meta = ('reviewText', 'string'))
    dask_df = dask_df.dropna(subset ='reviewText')
    
    #spellcheck
    dask_df['reviewText'] = dask_df['reviewText'].apply(fixSentence, meta = ('reviewText', 'string'))
    
    #clean_text
    dask_df['reviewText'] = dask_df['reviewText'].apply(lambda x: clean_text(x))

    #create target variable groupings
    #0 if negative, 1 if positive, 2 if neutral
    dask_df['pos_neg'] = dask_df['overall'].apply(lambda x: 1 if x >3 else (2 if x == 3 else 0))
    
    #0 if neutral, 1 if extreme, 2 if non-neutral
    dask_df['neutrality'] = dask_df['overall'].apply(lambda x: 1 if x == 1 or x == 5 else (0 if x == 3 else 2))
    
    #get num words
    token = RegexpTokenizer(r'[a-zA-Z0-9]+')
    dask_df['num_words'] = dask_df['reviewText'].apply(lambda x: len(token.tokenize(x)))

    
    dask_df = dask_df.compute()
    
    dd.to_csv(dask_df, saved_name, single_file = True)


In [ ]:
preprocess(elec, 'preprocessed_electronics.csv')
print('electronics preprocessed')
preprocess(allcats, 'preprocessed_allcats.csv')
print('allcats preprocessed')

In [ ]:
elec = pd.read_csv('preprocessed_electronics.csv')
allcats = pd.read_csv('preprocessed_allcats.csv')

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
def create_wordcloud_text(df, num_review_words):
    df = df[el_clean['num_words'] <= num_review_words]
    
    df_positive = df.loc[df['pos_neg'] == 1]
    df_negative = df.loc[df['pos_neg'] == 0]
    df_neutral = df.loc[df['pos_neg'] == 2]

    pos_text = " ".join(review for review in df_positive['reviewText'].astype(str))
    neg_text = " ".join(review for review in df_negative['reviewText'].astype(str))
    neut_text = " ".join(review for review in df_neutral['reviewText'].astype(str))

    return pos_text, neg_text, neut_text
    
stopwords = set(STOPWORDS)


### Electronics manual clean

In [ ]:
pos_text, neg_text, neut_text = create_wordcloud_text(elec, 1)

In [ ]:
poswordcloud = WordCloud(stopwords=stopwords, max_words=100, background_color="white").generate(pos_text)
plt.figure()
plt.imshow(poswordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Most Frequent Words for Positive Reviews with 1 word')
plt.show()

In [ ]:
negwordcloud = WordCloud(stopwords=stopwords, max_words=100, background_color="white").generate(neg_text)
plt.figure()
plt.imshow(negwordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Most Frequent Words for Negative Reviews with 1 word')
plt.show()

In [ ]:
neutwordcloud = WordCloud(stopwords=stopwords, max_words=100, background_color="white").generate(neut_text)
plt.figure()
plt.imshow(neutwordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Most Frequent Words for Neutral Reviews with 1 word')
plt.show()

In [ ]:
drop_indices = elec[(elec['num_words'] == 1) & (elec['reviewText'].str.contains('good')) & (elec['pos_neg'] == 0)].index
elec.drop(drop_indices, inplace=True)
drop_indices = elec[(elec['num_words'] == 1) & (elec['reviewText'].str.contains('great')) & (elec['pos_neg'] == 0)].index
elec.drop(drop_indices, inplace=True)

In [ ]:
elec.to_csv('electronics_final_allsamples.csv')

### Allcats manual clean

In [ ]:
pos_text, neg_text, neut_text = create_wordcloud_text(allcats, 1)

In [ ]:
poswordcloud = WordCloud(stopwords=stopwords, max_words=100, background_color="white").generate(pos_text)
plt.figure()
plt.imshow(poswordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Most Frequent Words for Positive Reviews with 1 word')
plt.show()

In [ ]:
negwordcloud = WordCloud(stopwords=stopwords, max_words=100, background_color="white").generate(neg_text)
plt.figure()
plt.imshow(negwordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Most Frequent Words for Negative Reviews with 1 word')
plt.show()

In [ ]:
neutwordcloud = WordCloud(stopwords=stopwords, max_words=100, background_color="white").generate(neut_text)
plt.figure()
plt.imshow(neutwordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Most Frequent Words for Neutral Reviews with 1 word')
plt.show()

In [ ]:
drop_indices = elec[(elec['num_words'] == 1) & (elec['reviewText'].str.contains('good')) & (elec['pos_neg'] == 0)].index
allcats.drop(drop_indices, inplace=True)
drop_indices = elec[(elec['num_words'] == 1) & (elec['reviewText'].str.contains('great')) & (elec['pos_neg'] == 0)].index
allcats.drop(drop_indices, inplace=True)

In [ ]:
allcats.to_csv('allcats_final_allsamples.csv')

# Downsampling

In [ ]:
def downsample_posneg_binary(df, save_path)
    drop_indices = df[df['pos_neg'] == 2].index
    df_posneg = df.drop(drop_indices, inplace = False)
    
    under_pn = RandomUnderSampler(sampling_strategy= 1 )
    x = df_posneg.drop(['overall', 'pos_neg', 'neutrality'], axis=1)
    y = df_posneg['pos_neg']

    x_under, y_under = under_pn.fit_resample(x, y)
    print(Counter(y_under))

    downsampled_pn_bin = x_under.join(y_under)
    downsampled_pn_bin.to_csv(save_path)

In [ ]:
downsample_posneg_binary(elec, 'electronics_final_20000samples.csv')
downsample_posneg_binary(elec, 'allcats_final_20000samples.csv')